In [1]:
!pip install flask pyngrok line-bot-sdk requests google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 11.3 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

NGROK_AUTHTOKEN = userdata.get("NGROK_AUTHTOKEN")
LINE_CHANNEL_ACCESS_TOKEN = userdata.get("LINE_CHANNEL_ACCESS_TOKEN")
LINE_CHANNEL_SECRET = userdata.get("LINE_CHANNEL_SECRET")
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

PORT = 5051

In [3]:
from pyngrok import ngrok

ngrok.kill()
ngrok.set_auth_token(NGROK_AUTHTOKEN)

tunnel = ngrok.connect(PORT)
WEBHOOK_URL = tunnel.public_url

print("ngrok url:", WEBHOOK_URL)

ngrok url: https://unseared-shante-unseraphically.ngrok-free.dev


In [4]:
import requests

def update_line_webhook(url):
    api = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {LINE_CHANNEL_ACCESS_TOKEN}",
        "Content-Type": "application/json"
    }
    res = requests.put(api, headers=headers, json={"endpoint": url})
    print("Webhook update:", res.status_code, res.text)

update_line_webhook(WEBHOOK_URL)

Webhook update: 200 {}


In [5]:
from google import genai
from google.genai.types import GenerateContentConfig

client = genai.Client(api_key=GEMINI_API_KEY)

# user_id -> Gemini chat session
user_chats = {}

def get_user_chat(user_id):
    if user_id not in user_chats:
        user_chats[user_id] = client.chats.create(
            model="gemini-2.5-flash",
            config=GenerateContentConfig(
                response_modalities=["TEXT"]
            )
        )
    return user_chats[user_id]

In [6]:
from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import MessageEvent, TextMessageContent

app = Flask(__name__)

configuration = Configuration(access_token=LINE_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(LINE_CHANNEL_SECRET)

In [7]:
@app.route("/", methods=["POST"])
def callback():
    signature = request.headers.get("X-Line-Signature")
    body = request.get_data(as_text=True)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return "OK"

In [8]:
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    user_id = event.source.user_id
    text = event.message.text

    if text.startswith("AI "):
        prompt = text[3:]
        chat = get_user_chat(user_id)
        reply_text = chat.send_message(prompt).text
    else:
        reply_text = text

    with ApiClient(configuration) as api_client:
        MessagingApi(api_client).reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=reply_text)]
            )
        )

In [9]:
app.run(port=PORT)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Jan/2026 07:16:30] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jan/2026 07:16:58] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jan/2026 07:17:05] "POST / HTTP/1.1" 200 -


In [10]:
ngrok.kill()